# CyTOFmerge vignette on the Vortex dataset

A step by step R documentation showing how to reproduce the results for the Vortex dataset

**Set the working directory to the 'Vortex' dataset folder**

In [1]:
setwd('Vortex Data')

**load the full (~800,000 cells) unannotated Vortex data**


FCS files as well as the cell population assignment can be downloaded from https://web.stanford.edu/~samusik/Panorama%20BM%201-10.zip

In [2]:
FCSfolder = 'FCS_files'
RelevantMarkers = c(9:47)

library(flowCore)
CyTOF.data = data.frame()
files = list.files(path = FCSfolder, pattern = '.fcs',full.names = TRUE)

  for (i in files){
    Temp <- read.FCS(i,transformation = FALSE, truncate_max_range = FALSE)
    colnames(Temp@exprs) <- Temp@parameters@data$desc
    CyTOF.data = rbind(CyTOF.data,as.data.frame(Temp@exprs)[,RelevantMarkers])
  }
VarNames = colnames(CyTOF.data)

# apply arcsinh transformation
CyTOF.data = asinh(CyTOF.data/5)

head(CyTOF.data)

Warning message:
"package 'flowCore' was built under R version 3.5.2"

Ter119,CD45.2,Ly6G,IgD,CD11c,F480,CD3,NKp46,CD23,CD34,...,CD150,CD25,TCRb,CD43,CD64,CD138,CD103,IgM,CD44,MHCII
-0.1086541461,1.4085079,-0.09079830,-0.01010285,1.49182892,1.31159406,-0.015352512,0.14489620,-0.04161210,-0.08590080,...,1.00997091,0.18240169,0.10066471,5.72158651,0.8232804,0.101529539,-0.03135053,-0.04920754,3.3600217,0.1486522
-0.1365840564,0.3853525,0.18922467,-0.13584190,-0.14589185,1.42239627,-0.126407113,-0.09836760,-0.01708075,-0.09644117,...,-0.10870177,-0.12409473,-0.04170852,0.48255284,0.6126746,0.221951512,-0.10842541,0.39402566,4.0050961,0.5267869
-0.0593331654,1.7997039,1.26799982,-0.14414045,-0.14605307,-0.07064983,-0.133813447,-0.15065636,-0.10419946,0.21037517,...,-0.04617730,0.06244335,-0.01316477,2.49379220,0.5817308,-0.122077208,-0.12280009,0.03264969,4.5221498,3.0288004
-0.1100932550,1.8107175,1.17020327,-0.07380096,0.05635457,-0.10861396,-0.001155501,-0.06392690,-0.05576744,-0.14791656,...,-0.03030463,-0.12035099,-0.04098447,-0.05359962,-0.1420399,0.009479272,0.06229437,5.11068182,0.5725295,3.6765001
-0.1492425385,2.0629390,0.02691357,-0.14170580,-0.14816851,2.05476011,-0.088010956,0.01288663,0.48578842,-0.09454828,...,-0.06671948,-0.02437281,-0.06912160,0.04409539,2.7926277,-0.041664907,0.17521665,0.16502512,4.4897235,0.8533346
-0.0002462804,1.8723901,-0.10477392,-0.12866120,-0.14900795,0.76899091,-0.118358227,0.73441443,-0.04733097,0.10290236,...,-0.10071110,0.06270636,1.23531204,3.56666233,0.8484111,0.297788876,0.18218071,-0.06804830,4.7893490,0.7667216


**Filter out highly correlated markers**

In [3]:
CorrThreshold = 0.8
# Filter highly correlated markers
R = cor(CyTOF.data,method = "pearson")
del = vector()
for (i in c(1:(dim(R)[1]-1))){
    for (j in c((i+1):dim(R)[1])){
        if(abs(R[i,j]) > CorrThreshold){
            Vi = var(CyTOF.data[,i])
            Vj = var(CyTOF.data[,j])
            if(Vi > Vj)
                del = c(del,j)
            else
                del = c(del, i)
            }
    }
}

del = unique(del)
Deleted.data = CyTOF.data[,del]
if(!pracma::isempty(del)){
    Deleted.markers = VarNames[del]
    CyTOF.data = CyTOF.data[,-del]
    VarNames = VarNames[-del]
    print('Remove by preprocessing')
    Deleted.markers
}else
    print('No markers removed by preprocessing')

[1] "No markers removed by preprocessing"


**Apply PCA markers selection**

In [4]:
rank = matrix(0,nrow = dim(CyTOF.data)[2], ncol = dim(CyTOF.data)[2])
  PCA = princomp(CyTOF.data)
  for (i in c(2:dim(CyTOF.data)[2])){
    Markers.importance = ((PCA$loadings^2)[,1:i]) %*% ((PCA$sdev^2)[1:i])
    Sorted.importance = sort(Markers.importance,decreasing = TRUE, index.return = TRUE)
    rank[,i] = Sorted.importance$ix
  }

# print out the 11 top markers
VarNames[rank[1:11,11]]

[1] "MHCII"   "Ly6C"    "CD11b"   "B220"    "IgM"     "CD44"    "CD16_32"
 [8] "Sca1"    "CD43"    "IgD"     "SiglecF"

**load the annotated (~500,000 cells) data**

A clean version can be found in 'VortexOrg.fcs', having an additional column called 'CSPLR_IDX' containing a unique id for each cell, which is used later for comparison between original and imputed data.

'VortexOrg.fcs' as well as all pre-computed files used later in this vignette can be downloaded from Flow Repository (http://flowrepository.org/id/FR-FCM-ZYVJ)

In [5]:
CyTOF.data = data.frame()
Temp <- read.FCS('VortexOrg.fcs',transformation = FALSE, truncate_max_range = FALSE)
colnames(Temp@exprs) <- Temp@parameters@data$desc
CyTOF.data = as.data.frame(Temp@exprs)[,c(1:40)]      # columns (1:39) contain markers, column 40 contains the unique cell id
VarNames = colnames(CyTOF.data)
Org.order <- order(CyTOF.data[,40])
CyTOF.data <- CyTOF.data[Org.order,]

uneven number of tokens: 457
The last keyword is dropped.
uneven number of tokens: 457
The last keyword is dropped.


In [6]:
head(CyTOF.data)

,Ter119,CD45.2,Ly6G,IgD,CD11c,F480,CD3,NKp46,CD23,CD34,...,CD25,TCRb,CD43,CD64,CD138,CD103,IgM,CD44,MHCII,CSPLR_IDX
18010,-0.32166645,1.2883543,-0.10073239,-0.3105940,-0.34924793,0.06786174,-0.31012473,-0.2959755,-0.2329866,-0.1080724820,...,-0.32002538,-0.2648633,3.5272372,0.298132300,-0.3258798,-0.3455625,-0.07602464,4.160683,0.5000296,1
7245,-0.30324870,1.5612683,-0.08132968,-0.1915980,-0.27889836,1.45096159,-0.05100589,0.8317363,-0.3461967,0.1350441277,...,-0.11114062,-0.3153106,1.2286713,-0.370737880,-0.1080197,-0.3696812,-0.31653252,3.810910,0.2304409,2
431598,-0.30153137,1.6992890,-0.27676046,-0.3565317,-0.23776907,1.36737084,-0.32148382,0.3494154,-0.3192171,0.0001553535,...,-0.32829195,-0.2761662,-0.2941253,0.801268399,-0.2181254,-0.3339199,0.54573721,4.337717,0.9224134,3
137800,-0.28222388,0.3448192,-0.05917977,-0.3417490,-0.30716118,1.44946277,-0.22664142,-0.2410803,-0.2906770,-0.1873397529,...,-0.23876508,0.0758084,3.0679810,0.022545358,-0.1275112,-0.2087217,-0.26546758,4.818168,0.7982297,4
486264,-0.02464599,2.5412021,0.36320993,-0.2386633,-0.22779606,1.77258217,-0.29112682,-0.3000089,-0.2163451,0.0385947004,...,-0.20791051,-0.0661618,-0.2185282,0.002612302,-0.2221171,-0.3446628,0.05941105,4.191411,0.3139563,5
157532,0.59202099,2.2582192,-0.34484372,3.6156151,0.07452723,1.08298922,-0.04192870,0.3481847,0.5488091,-0.1482390463,...,-0.02663678,-0.1938955,-0.3625352,0.121841371,-0.2690815,0.2199499,3.25255537,1.733557,5.3277197,6


In [7]:
# Apply Phenograph on the original data to obtain clusters, used to evaluate the cluster score

# library("Rphenograph")
# Output <- Rphenograph(CyTOF.data[,c(1:39)])
# Org_Clus <- factor(membership(Output[[2]]))
# Org_Clus <- as.matrix(Org_Clus)

# Org_Clus can be loaded from 'VortexORGLabels.csv' to save computation time
Org_Clus <- read.csv('VortexORGLabels.csv',header = TRUE)
Org_Clus <- as.vector(Org_Clus[,2])
Org_Clus <- Org_Clus[Org.order]

**Simulate the two panels and combine**

In [8]:
# The commented code below can produce the required result, however the imputed data for m=11 can be loaded from 'VortexIMP.fcs'
Imputed.data = data.frame()
Temp <- read.FCS('VortexIMP.fcs',transformation = FALSE, truncate_max_range = FALSE)
colnames(Temp@exprs) <- Temp@parameters@data$desc
Imputed.data = as.data.frame(Temp@exprs)[,c(1:40)]      # columns (1:39) contain markers, column 40 contains the unique cell id
Imp.order <- order(Imputed.data[,40])
Imputed.data <- Imputed.data[Imp.order,]
Imputed.data <- Imputed.data[,colnames(CyTOF.data)]     # put the imputed data columns in the same order as the original data

head(Imputed.data)

#####################################################################################################################
# X = sample(dim(CyTOF.data)[1],dim(CyTOF.data)[1])
# if((dim(CyTOF.data)[1]%%2)==0){
#   X1 = X[seq(1,length(X)-1,2)]
#   X2 = X[seq(2,length(X),2)]
# } else {
#   X1 = X[seq(1,length(X),2)]
#   X2 = X[seq(2,length(X)-1,2)]
# }
# 
# # here we show the results (scores) for 11 shared markers, but this m value can be changed to check different performances 
# m = 11  
# Data = CyTOF.data[,c(1:39)]
# # Simulate the two overlapping datasets
# Cut_Index = floor((dim(Data)[2]-m)/2)+m
# Data1 = as.matrix(Data[X1,c(rank[1:m,m],rank[(m+1):Cut_Index,m])])
# Data2 = as.matrix(Data[X2,c(rank[1:m,m],rank[(Cut_Index+1):dim(Data)[2],m])])
# Data.Sorted = as.matrix(Data[c(X1,X2),rank[,m]])
# 
# Subset.labels.1 = Org_Clus[X1]
# Subset.labels.2 = Org_Clus[X2]
# Subset.labels.Sorted = Org_Clus[c(X1,X2)]
# 
# # Find the 50 neighbors from one dataset to the other
# IDX1 = FNN::get.knnx(Data2[,1:m],Data1[,1:m], k = 50, algorithm = "kd_tree")
# IDX1 = IDX1$nn.index
# IDX2 = FNN::get.knnx(Data1[,1:m],Data2[,1:m], k = 50, algorithm = "kd_tree")
# IDX2 = IDX2$nn.index
# 
# Data.combine.1 = matrix(0,nrow = dim(Data1)[1],ncol = dim(Data.Sorted)[2])
# Data.combine.2 = matrix(0,nrow = dim(Data2)[1],ncol = dim(Data.Sorted)[2])
# 
# Subset.labels.combine.1 = as.vector(matrix(0,nrow = dim(Data1)[1]))
# Subset.labels.combine.2 = as.vector(matrix(0,nrow = dim(Data2)[1]))
# 
# getmode <- function(x) {
#   uniqx <- unique(x)
#   uniqx[which.max(tabulate(match(x, uniqx)))]
# }
# 
# # Combine datasets
# for (i in c(1:dim(Data1)[1])){
#   Data.combine.1[i,1:m] = Data1[i,1:m]
#   Data.combine.1[i,(m+1):Cut_Index] = Data1[i,(m+1):dim(Data1)[2]]
#   Data.combine.1[i,(Cut_Index+1):dim(Data.combine.1)[2]] = apply(Data2[IDX1[i,],(m+1):dim(Data2)[2]],2,median)
#   Subset.labels.combine.1[i] = getmode(Subset.labels.2[IDX1[i,]])
# }
# 
# for (i in c(1:dim(Data2)[1])){
#   Data.combine.2[i,1:m] = Data2[i,1:m]
#   Data.combine.2[i,(m+1):Cut_Index] = apply(Data1[IDX2[i,],(m+1):dim(Data1)[2]],2,median)
#   Data.combine.2[i,(Cut_Index+1):dim(Data.combine.2)[2]] = Data2[i,(m+1):dim(Data2)[2]]
#   Subset.labels.combine.2[i] = getmode(Subset.labels.1[IDX2[i,]])
# }
# 
# Data.combine = rbind(Data.combine.1,Data.combine.2)
# Subset.labels.combine = c(Subset.labels.combine.1,Subset.labels.combine.2)
#####################################################################################################################

uneven number of tokens: 457
The last keyword is dropped.
uneven number of tokens: 457
The last keyword is dropped.


,Ter119,CD45.2,Ly6G,IgD,CD11c,F480,CD3,NKp46,CD23,CD34,...,CD25,TCRb,CD43,CD64,CD138,CD103,IgM,CD44,MHCII,CSPLR_IDX
392884,-0.2500487,1.2883543,0.15371709,-0.3105940,-0.2749919,0.06786174,-0.31012473,-0.003436979,-0.2517234,-0.1080724820,...,-0.1825407,-0.2648633,3.5272372,0.298132300,-0.1018325,-0.2497950,-0.07602464,4.160683,0.5000296,1
17751,-0.2870310,1.5612683,0.11848408,-0.1915980,-0.2476686,1.45096159,-0.05100589,-0.234368533,-0.2488589,0.1350441277,...,-0.2613057,-0.3153106,1.2286713,-0.370737880,-0.2171743,-0.2308543,-0.31653252,3.810910,0.2304409,2
208751,-0.2906216,1.6992890,-0.03444875,-0.3565317,-0.2589885,1.36737084,-0.32148382,-0.232168943,-0.2654646,0.0001553535,...,-0.2478615,-0.2761662,-0.2941253,0.801268399,-0.2034232,-0.2862903,0.54573721,4.337717,0.9224134,3
350314,-0.2736817,0.3448192,0.05676027,-0.3417490,-0.2509668,1.44946277,-0.22664142,-0.118308306,-0.2415421,-0.1873397529,...,-0.2212926,0.0758084,3.0679810,0.022545358,-0.1286017,-0.2139755,-0.26546758,4.818168,0.7982297,4
203625,-0.2645330,2.5412021,-0.08398499,-0.2386633,-0.2574029,1.77258217,-0.29112682,-0.227561668,-0.2599653,0.0385947004,...,-0.2498236,-0.0661618,-0.2185282,0.002612302,-0.1216073,-0.2389690,0.05941105,4.191411,0.3139563,5
100933,-0.2722035,2.2582192,0.24273992,3.6156151,-0.1991128,1.08298922,-0.04192870,-0.160588458,0.1052791,-0.1482390463,...,-0.2420946,-0.1938955,-0.3625352,0.121841371,-0.1692547,-0.1860164,3.25255537,1.733557,5.3277197,6


## Calculate scores

1) Distance score

In [9]:
# In order to calculate the Distance score, we need to calculate the average random Euclidean distance in the original data
# The commented code below can produce the result, but keep in mind it takes long time to calculate all the pairwise distances
# The pre-computed distances can be loaded from 'Euc_Dist_Avg.csv'
Euc_Dist_Avg <- read.csv('Euc_Dist_Avg.csv', sep = "", dec = ",",header = FALSE)
Euc_Dist_Avg <- as.vector(Euc_Dist_Avg[,1])

#####################################################################################################################
# DownSampleSize = 100000
# Data = CyTOF.data[,1:39]
# Temp.Data = Data[sample(dim(Data)[1],DownSampleSize),]
  
# # Calculate the average pairwise Euclidean distance (takes long time)
# Euc_Dist = as.vector(matrix(0,nrow = dim(Data)[1]))
# for (i in c(1:dim(Data)[1])){
#     for (j in c(i:dim(Data)[1])){
#         Euc_Dist[i]=Euc_Dist[i]+sqrt(sum((Data[i,] - Data[j,])^2))
#         Euc_Dist[j]=Euc_Dist[j]+sqrt(sum((Data[i,] - Data[j,])^2))
#     }
# }
# Euc_Dist_Avg = Euc_Dist/dim(Data)[1]
#####################################################################################################################

# Now we calculate the Euclidean distances between the original and imputed datasets
euc_dist = sqrt(rowSums((CyTOF.data[,c(1:39)]-Imputed.data[,c(1:39)])^2))
    
Distance_Score = (median(Euc_Dist_Avg)-median(euc_dist))/median(Euc_Dist_Avg)

print(paste("Distance Score =", Distance_Score*100, "%"))

[1] "Distance Score = 84.0094398450122 %"


2) Nearest Neighbor score

In [10]:
# In order to calculate the Nearest Neighbor score, we need to calculate the distance to the 1st nearest-neighbor for every
# cell in the original data
# The commented code below can produce the result, but keep in mind it takes long time to calculate all the pairwise distances
# in order to find the smallest distance for every cell
# The pre-computed distances can be loaded from 'NN_distance.csv'
NN_dist <- read.csv('NN_distance.csv', sep = "", dec = ",",header = FALSE)
NN_dist <- as.vector(NN_dist[,1])

#####################################################################################################################
# NN = FNN::get.knn(CyTOF.data[,c(1:39)], k = 1, algorithm = "kd_tree")
# NN_dist = NN$nn.dist
#####################################################################################################################

NN_Score = sum(NN_dist > euc_dist)/length(euc_dist)

print(paste("Nearest Neighbor Score =", NN_Score*100, "%"))


[1] "Nearest Neighbor Score = 82.0531662992383 %"


3) Cluster score 'Adjusted rand index'

In [11]:
# Apply Phenograph on the imputed data to obtain new clustering, to be compared with the original clustering

# library("Rphenograph")
# Output <- Rphenograph(Imputed.data[,c(1:39)])
# Imp_Clus <- factor(membership(Output[[2]]))
# Imp_Clus <- as.matrix(Imp_Clus)

# Imp_Clus can be loaded from 'VortexIMPLabels.csv' to save computation time
Imp_Clus <- read.csv('VortexIMPLabels.csv',header = TRUE)
Imp_Clus <- as.vector(Imp_Clus[,2])
Imp_Clus <- Imp_Clus[Imp.order]

# Calculate the Adjusted rand index between original and imputed clustering
ARI = mclust::adjustedRandIndex(Org_Clus,Imp_Clus)

print(paste("Adjusted Rand index =", ARI))

[1] "Adjusted Rand index = 0.895390569356364"
